# Build llcuda v2.0 for Tesla T4 (Google Colab)

**Purpose**: Build complete CUDA 12 binaries for llcuda v2.0 on Google Colab Tesla T4 GPU

**Output**:
1. llama.cpp binaries (264 MB) - HTTP server mode
2. llcuda_cpp.so (native extension) - v2.0 Tensor API

**Requirements**:
- Google Colab with T4 GPU
- CUDA 12.x
- Python 3.11+

---

## Step 1: Verify GPU and Environment

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,compute_cap,driver_version,memory.total --format=csv

In [ ]:
# Verify CUDA version
!nvcc --version

In [ ]:
# Check Python version
import sys
print(f"Python: {sys.version}")
print(f"Expected: 3.10+ (Colab default)")

In [ ]:
# Verify compute capability
import subprocess

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=compute_cap', '--format=csv,noheader'],
    capture_output=True,
    text=True
)
compute_cap = result.stdout.strip()
major, minor = map(int, compute_cap.split('.'))

print(f"Compute Capability: SM {major}.{minor}")

if major == 7 and minor == 5:
    print("✓ Tesla T4 detected - Perfect for llcuda v2.0!")
elif major >= 7 and minor >= 5:
    print(f"✓ SM {major}.{minor} detected - Compatible with llcuda v2.0")
else:
    print(f"⚠ WARNING: SM {major}.{minor} is below SM 7.5 (T4)")
    print("llcuda v2.0 requires SM 7.5+ for Tensor Cores and FlashAttention")

## Step 2: Clone llcuda v2.0 Repository

In [ ]:
# Clone llcuda v2.0
!git clone https://github.com/waqasm86/llcuda.git
%cd llcuda

In [ ]:
# Verify we have llcuda v2.0 structure
import os
from pathlib import Path

print("Checking llcuda v2.0 repository structure...")
print("=" * 60)

# Required files for llcuda v2.0
required_files = [
    'CMakeLists.txt',
    'csrc/core/device.h',
    'csrc/core/device.cu',
    'csrc/core/tensor.h',
    'csrc/core/tensor.cu',
    'csrc/bindings.cpp',
    'csrc/ops/matmul.h',
    'csrc/ops/matmul.cu',
    'llcuda/__init__.py',
    'llcuda/_internal/bootstrap.py',
    'pyproject.toml',
]

missing_files = []
found_files = []

for file in required_files:
    if os.path.exists(file):
        print(f"✓ {file}")
        found_files.append(file)
    else:
        print(f"✗ MISSING: {file}")
        missing_files.append(file)

print("=" * 60)
print(f"\nFound: {len(found_files)}/{len(required_files)} files")

if missing_files:
    print(f"\n❌ ERROR: {len(missing_files)} required files are missing!")
    print("\nMissing files:")
    for file in missing_files:
        print(f"  - {file}")
    print("\nPossible causes:")
    print("  1. Repository clone incomplete")
    print("  2. Wrong branch (need 'main' branch)")
    print("  3. Files not yet pushed to GitHub")
    print("\nSolution:")
    print("  1. Delete the llcuda directory: !rm -rf /content/llcuda")
    print("  2. Re-clone: !git clone https://github.com/waqasm86/llcuda.git")
    print("  3. Ensure you're on main branch: !git checkout main")
    raise FileNotFoundError(f"Required llcuda v2.0 files not found: {', '.join(missing_files)}")

print("\n✅ All required files present - Ready to build!")

# Show directory structure
print("\nDirectory structure:")
!ls -la csrc/
!ls -la csrc/core/
!ls -la csrc/ops/

## Step 3: Build llama.cpp Binaries (HTTP Server Mode)

These binaries power the v1.x HTTP server mode and GGUF model support.

In [ ]:
# Clone llama.cpp
%cd /content
!git clone https://github.com/ggml-org/llama.cpp.git
%cd llama.cpp

In [ ]:
# Configure llama.cpp for Tesla T4 with FlashAttention
!cmake -B build_cuda12_t4 \
    -DCMAKE_BUILD_TYPE=Release \
    -DGGML_CUDA=ON \
    -DCMAKE_CUDA_ARCHITECTURES="75" \
    -DCMAKE_CUDA_COMPILER=/usr/local/cuda/bin/nvcc \
    -DGGML_NATIVE=OFF \
    -DGGML_CUDA_FORCE_MMQ=OFF \
    -DGGML_CUDA_FORCE_CUBLAS=OFF \
    -DGGML_CUDA_FA=ON \
    -DGGML_CUDA_FA_ALL_QUANTS=ON \
    -DGGML_CUDA_GRAPHS=ON \
    -DGGML_CUDA_PEER_MAX_BATCH_SIZE=128 \
    -DLLAMA_BUILD_SERVER=ON \
    -DLLAMA_BUILD_TOOLS=ON \
    -DLLAMA_CURL=ON \
    -DBUILD_SHARED_LIBS=ON \
    -DCMAKE_INSTALL_RPATH='$ORIGIN/../lib' \
    -DCMAKE_BUILD_WITH_INSTALL_RPATH=ON

In [ ]:
# Build (this takes ~10 minutes)
import time

print("Building llama.cpp... (estimated 10 minutes)")
start_time = time.time()

!cmake --build build_cuda12_t4 --config Release -j$(nproc)

elapsed = time.time() - start_time
print(f"\n✓ Build completed in {elapsed/60:.1f} minutes")

In [ ]:
# Verify binaries were built
!ls -lh build_cuda12_t4/bin/llama-server
!ls -lh build_cuda12_t4/bin/*.so* | head -20

In [ ]:
# Test llama-server (fix libcuda.so.1 issue)
import os
import subprocess

# Find libcuda.so.1
!find /usr -name "libcuda.so.1" 2>/dev/null | head -1

# Set LD_LIBRARY_PATH and test
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/targets/x86_64-linux/lib:/content/llama.cpp/build_cuda12_t4/bin'

result = subprocess.run(
    ['/content/llama.cpp/build_cuda12_t4/bin/llama-server', '--version'],
    env=os.environ,
    capture_output=True,
    text=True
)

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print("\n✓ llama-server works!")
else:
    print(f"\n✗ Error: Return code {result.returncode}")

In [ ]:
# Package llama.cpp binaries
%cd /content

!mkdir -p package_t4/bin
!mkdir -p package_t4/lib

# Copy essential binaries
!cp llama.cpp/build_cuda12_t4/bin/llama-server package_t4/bin/
!cp llama.cpp/build_cuda12_t4/bin/llama-cli package_t4/bin/
!cp llama.cpp/build_cuda12_t4/bin/llama-quantize package_t4/bin/
!cp llama.cpp/build_cuda12_t4/bin/llama-embedding package_t4/bin/
!cp llama.cpp/build_cuda12_t4/bin/llama-bench package_t4/bin/

# Copy all shared libraries
!cp llama.cpp/build_cuda12_t4/bin/*.so* package_t4/lib/

print("\n=== Package Contents ===")
!du -sh package_t4
!du -sh package_t4/bin
!du -sh package_t4/lib

## Step 4: Build llcuda v2.0 Native Extension (Tensor API)

This is the NEW v2.0 PyTorch-style tensor API with custom CUDA kernels.

In [ ]:
# Install dependencies
!pip install -q pybind11 cmake ninja
!pip install -q numpy torch --upgrade

In [ ]:
# Build llcuda v2.0 native extension
%cd /content/llcuda

# Clean previous builds
!rm -rf build/native_t4
!rm -f llcuda_cpp*.so

# Create build directory
!mkdir -p build/native_t4
%cd build/native_t4

In [ ]:
# Configure CMake for T4
!cmake ../.. \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_CUDA_ARCHITECTURES="75" \
    -DPython3_EXECUTABLE=$(which python3)

In [ ]:
# Build (this takes ~5 minutes)
print("Building llcuda v2.0 native extension... (estimated 5 minutes)")
start_time = time.time()

!make -j$(nproc)

elapsed = time.time() - start_time
print(f"\n✓ Build completed in {elapsed/60:.1f} minutes")

In [ ]:
# Verify the extension was built
!ls -lh llcuda_cpp*.so
!file llcuda_cpp*.so

In [ ]:
# Copy extension to package root
!cp llcuda_cpp*.so /content/llcuda/

print("\n=== Extension Info ===")
!ls -lh /content/llcuda/llcuda_cpp*.so
!du -sh /content/llcuda/llcuda_cpp*.so

In [ ]:
# Test the native extension
%cd /content/llcuda

import sys
sys.path.insert(0, '/content/llcuda')

try:
    import llcuda_cpp
    
    # Test device detection
    device_count = llcuda_cpp.Device.get_device_count()
    print(f"✓ Devices found: {device_count}")
    
    # Test device properties
    props = llcuda_cpp.Device.get_device_properties(0)
    print(f"✓ Device: {props.name}")
    print(f"✓ Compute: SM {props.compute_capability_major}.{props.compute_capability_minor}")
    print(f"✓ Memory: {props.total_memory / 1024**3:.2f} GB")
    
    # Test tensor creation
    tensor = llcuda_cpp.Tensor([10, 10], llcuda_cpp.DType.Float32, 0)
    print(f"✓ Tensor created: {tensor.shape()}")
    
    # Test matrix multiplication
    A = llcuda_cpp.Tensor.zeros([64, 64], llcuda_cpp.DType.Float32, 0)
    B = llcuda_cpp.Tensor.zeros([64, 64], llcuda_cpp.DType.Float32, 0)
    C = llcuda_cpp.ops.matmul(A, B)
    print(f"✓ MatMul works: {C.shape()}")
    
    print("\n✓ llcuda v2.0 native extension works perfectly!")
    
except Exception as e:
    print(f"\n✗ Error testing extension: {e}")
    import traceback
    traceback.print_exc()

## Step 5: Create Release Packages

In [ ]:
# Package 1: llama.cpp binaries (HTTP server mode)
%cd /content

!tar -czf llcuda-binaries-cuda12-t4.tar.gz package_t4/

print("\n=== Package 1: llama.cpp Binaries ===")
!ls -lh llcuda-binaries-cuda12-t4.tar.gz
!du -h llcuda-binaries-cuda12-t4.tar.gz

In [ ]:
# Package 2: llcuda v2.0 native extension
%cd /content/llcuda

# Create package directory
!mkdir -p native_extension_t4
!cp llcuda_cpp*.so native_extension_t4/
!cp CMakeLists.txt native_extension_t4/
!cp build_native.sh native_extension_t4/

# Create metadata
!echo 'Tesla T4 (SM 7.5)' > native_extension_t4/GPU_TARGET.txt
!echo 'CUDA 12.x' >> native_extension_t4/GPU_TARGET.txt
!echo 'Built on Google Colab' >> native_extension_t4/GPU_TARGET.txt
!date >> native_extension_t4/GPU_TARGET.txt

!tar -czf llcuda-v2-native-t4.tar.gz native_extension_t4/

print("\n=== Package 2: llcuda v2.0 Native Extension ===")
!ls -lh llcuda-v2-native-t4.tar.gz
!du -h llcuda-v2-native-t4.tar.gz

In [ ]:
# Package 3: Combined package (both binaries + extension)
%cd /content

!mkdir -p llcuda_v2_complete_t4/binaries
!mkdir -p llcuda_v2_complete_t4/native

# Copy llama.cpp binaries
!cp -r package_t4/* llcuda_v2_complete_t4/binaries/

# Copy native extension
!cp llcuda/llcuda_cpp*.so llcuda_v2_complete_t4/native/

# Create README
cat > llcuda_v2_complete_t4/README.md << 'EOF'
# llcuda v2.0 Complete Package for Tesla T4

**Built on**: Google Colab
**GPU**: Tesla T4 (SM 7.5)
**CUDA**: 12.x

## Contents

### binaries/
llama.cpp binaries for HTTP server mode:
- `bin/llama-server` - HTTP inference server
- `bin/llama-cli` - Command-line interface
- `bin/llama-quantize` - Model quantization tool
- `lib/*.so` - Shared libraries (libggml-cuda.so with FlashAttention)

### native/
llcuda v2.0 native extension:
- `llcuda_cpp.cpython-*.so` - PyTorch-style Tensor API

## Installation

```bash
# Extract
tar -xzf llcuda_v2_complete_t4.tar.gz

# Copy to llcuda package
cp -r llcuda_v2_complete_t4/binaries/* ~/llcuda/binaries/cuda12/
cp llcuda_v2_complete_t4/native/*.so ~/llcuda/
```

## Features

- ✅ FlashAttention enabled
- ✅ Tensor Core optimizations
- ✅ CUDA Graphs support
- ✅ All quantization formats (Q4_K_M, Q8_0, NF4, etc.)
- ✅ Optimized for SM 7.5+ GPUs

EOF

!tar -czf llcuda-v2-complete-t4.tar.gz llcuda_v2_complete_t4/

print("\n=== Package 3: Complete Bundle ===")
!ls -lh llcuda-v2-complete-t4.tar.gz
!du -h llcuda-v2-complete-t4.tar.gz

## Step 6: Summary and Download

In [ ]:
# Display all packages
print("\n" + "="*60)
print("BUILD COMPLETE - llcuda v2.0 for Tesla T4")
print("="*60)

print("\n📦 Available Packages:")
print("-" * 60)

packages = [
    "/content/llcuda-binaries-cuda12-t4.tar.gz",
    "/content/llcuda/llcuda-v2-native-t4.tar.gz",
    "/content/llcuda-v2-complete-t4.tar.gz"
]

for pkg in packages:
    !ls -lh {pkg}

print("\n✅ All packages built successfully!")
print("\n📥 Download instructions:")
print("   1. Click the folder icon on the left")
print("   2. Right-click on the .tar.gz files")
print("   3. Select 'Download'")
print("\n   Or run the cell below to auto-download")

In [ ]:
# Auto-download all packages
from google.colab import files

print("Downloading Package 1: llama.cpp binaries (264 MB)...")
files.download('/content/llcuda-binaries-cuda12-t4.tar.gz')

print("\nDownloading Package 2: llcuda v2.0 native extension...")
files.download('/content/llcuda/llcuda-v2-native-t4.tar.gz')

print("\nDownloading Package 3: Complete bundle...")
files.download('/content/llcuda-v2-complete-t4.tar.gz')

print("\n✓ All downloads started!")

## Step 7: Upload to GitHub Releases (Optional)

In [ ]:
# Install GitHub CLI (if you want to upload directly)
!curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null
!sudo apt update
!sudo apt install gh -y

In [ ]:
# Authenticate with GitHub (manual step)
print("Run this command and follow the prompts:")
print("  gh auth login")
print("\nThen run the upload commands below")

In [ ]:
# Upload to GitHub releases (after authentication)
# Uncomment and modify these commands:

# !gh release create v2.0.0 \
#     --repo waqasm86/llcuda \
#     --title "llcuda v2.0.0 - Tesla T4 Release" \
#     --notes "Complete CUDA 12 build for Tesla T4 (SM 7.5)" \
#     /content/llcuda-binaries-cuda12-t4.tar.gz \
#     /content/llcuda/llcuda-v2-native-t4.tar.gz \
#     /content/llcuda-v2-complete-t4.tar.gz

print("Upload commands ready (uncomment to use)")

## 🎉 Build Complete!

You now have:

1. **llcuda-binaries-cuda12-t4.tar.gz** (264 MB)
   - llama.cpp server with FlashAttention
   - For HTTP server mode (v1.x compatibility)

2. **llcuda-v2-native-t4.tar.gz** (~100 MB)
   - llcuda v2.0 native extension
   - PyTorch-style Tensor API

3. **llcuda-v2-complete-t4.tar.gz** (~350 MB)
   - Everything bundled together

### Next Steps:

1. Download packages to your local machine
2. Upload to GitHub releases:
   ```bash
   gh release create v2.0.0 \
       --repo waqasm86/llcuda \
       --title "llcuda v2.0.0 - Tesla T4 Release" \
       llcuda-*.tar.gz
   ```

3. Update bootstrap.py to download from releases

4. Test on fresh Colab instance

---

**Built with**: Google Colab Tesla T4 | CUDA 12 | Python 3.10+
**For**: llcuda v2.0 - Unsloth Integration
